In [35]:
import gzip
import numpy as np
import urllib
import matplotlib.pyplot as plt
import random
import datetime
import pandas as pd
import surprise
from tqdm import tqdm_notebook
from surprise import Dataset, Reader, accuracy, SVD, BaselineOnly
from collections import defaultdict, Counter
from surprise.model_selection import train_test_split

In [2]:
def readGz(f):
  for l in tqdm_notebook(gzip.open(f), total=1689188):
    yield eval(l)
data = list(readGz("reviews_Electronics_5.json.gz"))

In [3]:
total = 1689188
train_length = 800000
train_data = pd.DataFrame(data[:train_length])
test_data = pd.DataFrame(data[train_length:])

reader = Reader(rating_scale=(1, 5))
test_dataset = Dataset.load_from_df(test_data[['reviewerID', 'asin', 'overall']], reader)
_, testset = train_test_split(test_dataset, test_size=len(test_data)-1)
test_dataset = Dataset.load_from_df(test_data[['reviewerID', 'asin', 'overall']], reader)

In [4]:
# using SVD algorithm to predict the rating.
# r^ui=μ+bu+bi+qTipu
# https://surprise.readthedocs.io/en/stable/matrix_factorization.html
trainset = train_dataset.build_full_trainset()
algo = surprise.prediction_algorithms.matrix_factorization.SVD()
algo.fit(trainset)

In [31]:
predictions = algo.test(testset)

In [34]:
print("the rmse for algorithm SVD in test set is: ", accuracy.rmse(predictions))

RMSE: 1.1401
the rmse in train set is:  1.1400902133448116


In [36]:
# r^ui=bui=μ+bu+bi
# the baseline in surprise.
# I thought the rmse should be the same as the baseline algo written by us.
# but it don't.maybe caused by parameters?
base_algo = BaselineOnly()
base_algo.fit(trainset)
base_predictions = base_algo.test(testset)
print("the rmse for algorithm base_line in test set is: ", accuracy.rmse(base_predictions))

Estimating biases using als...
RMSE: 1.1441
the rmse for algorithm base_line in test set is:  1.1441399018872118
